### First implementation test

In [45]:
import open3d as o3d, numpy as np

def level_cloud(pcd: o3d.geometry.PointCloud,
                distance_thresh=0.002,          # 2 mm plane tolerance
                num_iters=1000):
    """Return a copy of `pcd` rotated so the dominant plane is horizontal."""
    # 1. RANSAC: n·x + d = 0
    (a, b, c, d), inliers = pcd.segment_plane(distance_thresh,
                                              ransac_n=3,
                                              num_iterations=num_iters)
    n = np.array([a, b, c])
    n /= np.linalg.norm(n)

    # 2. Build rotation that sends n → [0,0,1]
    z = np.array([0, 0, 1])
    v = np.cross(n, z)
    s = np.linalg.norm(v)
    if s < 1e-6:                       # already level
        R = np.eye(3)
    else:
        c = np.dot(n, z)
        vx = np.array([[   0, -v[2],  v[1]],
                       [ v[2],    0, -v[0]],
                       [-v[1],  v[0],   0]])
        R = np.eye(3) + vx + vx @ vx * ((1 - c) / (s ** 2))

    # 3. Rotate **around the plane centroid** to keep the table at Z≈const
    centroid = pcd.points[inliers[0]]          # any point on the plane
    pcd_level = pcd.translate(-centroid)       # move plane to origin
    pcd_level.rotate(R, center=(0, 0, 0))      # level it
    pcd_level.translate(centroid)              # move back
    return pcd_level, R, d

def measure_box_mm(pcd_path: str, skew_ok=True,
                   plane_thresh_m=0.005,      # in metres  (5 mm)
                   min_points=20,
                   mm_to_m = 1/1000.0):
    import open3d as o3d, numpy as np

    pcd = o3d.io.read_point_cloud(pcd_path)
    if len(pcd.points) == 0:
        raise ValueError("PCD is empty or path is wrong")

    # ---- 0.  RESCALE TO METRES -------------------------------------------
    pts_m = np.asarray(pcd.points, dtype=np.float64) * mm_to_m
    pcd.points = o3d.utility.Vector3dVector(pts_m)

    # ---- 1.  LEVEL SCENE --------------------------------------------------
    pcd, _, _ = level_cloud(pcd, distance_thresh=plane_thresh_m)

    # ---- 2.  REMOVE TABLE -------------------------------------------------
    pcd, _    = pcd.remove_statistical_outlier(20, 2.0)
    _, inl    = pcd.segment_plane(plane_thresh_m, 3, 1000)
    obj       = pcd.select_by_index(inl, invert=True)
    if len(obj.points) == 0:
        raise RuntimeError("Plane removal ate the whole cloud – widen plane_thresh_m")

    # ---- 3.  PICK BOX CLUSTER --------------------------------------------
    nn_d   = obj.compute_nearest_neighbor_distance()
    eps    = 2 * np.percentile(nn_d, 90)        # still in metres
    labels = np.array(obj.cluster_dbscan(float(eps), min_points, False))

    if np.any(labels >= 0):
        biggest = np.bincount(labels[labels >= 0]).argmax()
        obj     = obj.select_by_index(np.where(labels == biggest)[0])
    else:
        print("DBSCAN found no clusters; using all remaining points")

    # ---- 4.  MEASURE ------------------------------------------------------
    bbox = (obj.get_oriented_bounding_box() if skew_ok
            else obj.get_axis_aligned_bounding_box())
    dims_m = bbox.extent                      # metres
    dims_cm = dims_m * 100                  # back to cm for display

    return dims_cm, obj, bbox                # cm → easier to read

# ---- Run once ----
dims_cm, cloud, bb = measure_box_mm("PointCloud/PointCloud_box_a1.pcd")
print(f"Box dimensions ≈ {dims_cm[0]:.1f} × {dims_cm[1]:.1f} × {dims_cm[2]:.1f} cm")

# Optional sanity check in a viewer
o3d.visualization.draw_geometries([cloud.paint_uniform_color([0.1,0.7,0.8]),
                                   bb])


Box dimensions ≈ 20.5 × 10.8 × 1.0 cm


### Height fix attempt

In [62]:
import open3d as o3d, numpy as np

MM_TO_M = 1/1000.0           # set to 1.0 if your file is already in metres

# ---------------------------------------------------------------------
def plane_fit(cloud, thresh, n_iter=1000):
    (a, b, c, d), inl = cloud.segment_plane(thresh, 3, n_iter)
    n = np.array([a, b, c], dtype=float)
    n /= np.linalg.norm(n)
    d /= np.linalg.norm([a, b, c])
    return n, d, inl
# ---------------------------------------------------------------------

def measure_box_planes(pcd_path,
                       table_thresh_mm=5, lid_thresh_mm=3,
                       min_points=500):
    # 0. LOAD & RESCALE ----------------------------------------------------
    pcd  = o3d.io.read_point_cloud(pcd_path)
    pcd.points = o3d.utility.Vector3dVector(
        np.asarray(pcd.points, dtype=np.float64) * MM_TO_M)

    table_thr = table_thresh_mm * MM_TO_M
    lid_thr   = lid_thresh_mm   * MM_TO_M

    # 1. TABLE PLANE -------------------------------------------------------
    n_tab, d_tab, inl_tab = plane_fit(pcd, table_thr)
    table_cloud = pcd.select_by_index(inl_tab)
    obj_cloud   = pcd.select_by_index(inl_tab, invert=True)

    # 2. KEEP POINTS <=15 cm ABOVE TABLE -----------------------------------
    pts      = np.asarray(obj_cloud.points)              # ★ FIX ①
    mask     = pts.dot(n_tab) > -d_tab - 0.15            # 0.15 m gate
    obj_cloud = obj_cloud.select_by_index(np.where(mask)[0])

    # 3. LARGEST DBSCAN CLUSTER  ------------------------------------------
    labels = np.array(obj_cloud.cluster_dbscan(eps=0.02, min_points=50))
    if np.any(labels >= 0):
        biggest = np.bincount(labels[labels >= 0]).argmax()
        main    = obj_cloud.select_by_index(np.where(labels == biggest)[0])
    else:
        raise RuntimeError("No cluster found – tweak eps/min_points.")

    # 4. LID/TOP PLANE -----------------------------------------------------
    n_lid, d_lid, inl_lid = plane_fit(main, lid_thr)
    lid_cloud = main.select_by_index(inl_lid)

    # 5. DIMENSIONS --------------------------------------------------------
    height_m = abs(d_lid - d_tab)                        # planes share n
    bbox_xy  = main.get_oriented_bounding_box().extent[:2]

    dims_mm  = np.r_[bbox_xy * 100, height_m * 100]    # ★ FIX ②
    return dims_mm, main, lid_cloud, table_cloud
# ---------------------------------------------------------------------

# Example call --------------------------------------------------------
dims_mm, parcel, lid_pts, table_pts = measure_box_planes(
        "PointCloud/PointCloud_box_a4.pcd")

print(f"≈ {dims_mm[0]:.1f} cm × {dims_mm[1]:.1f} cm × {dims_mm[2]:.1f} cm  (L×W×H)")

# Visualization --------------------------------------------------------
# paint each set a distinct color
table_pts.paint_uniform_color([1.0, 0.0, 0.0])    # red -> table
lid_pts.paint_uniform_color([0.0, 1.0, 0.0])      # green -> top of box
parcel.paint_uniform_color([0.2, 0.8, 1.0])       # cyan -> in between

# show them all together
o3d.visualization.draw_geometries([
    table_pts,
    lid_pts,
    parcel
], 
window_name="Table (red) – Lid (green) – Parcel (cyan)",
width=800, height=600)

≈ 27.7 cm × 15.1 cm × 10.5 cm  (L×W×H)


### Saving point cloud

In [63]:
import numpy as np

# 1) sanity‐check counts
print("table_pts:", len(table_pts.points))
print("lid_pts:  ", len(lid_pts.points))
print("parcel:   ", len(parcel.points))

# 2) merge them
combined = table_pts + lid_pts + parcel
print("combined:", len(combined.points), "points")

# 3) (optional) if you want colors and some viewers choke on rgb,
#    paint everything white so at least the points show up:
combined.paint_uniform_color([1.0, 1.0, 1.0])

# 4) write an ASCII PCD
o3d.io.write_point_cloud("segmented_box_ascii.pcd",
                         combined,
                         write_ascii=True)
print("Wrote segmented_box_ascii.pcd as ASCII, should be viewable now.")

table_pts: 209979
lid_pts:   15448
parcel:    27365
combined: 252792 points
Wrote segmented_box_ascii.pcd as ASCII, should be viewable now.


In [64]:
combined_mm = table_pts + lid_pts + parcel          # already merged
combined_mm.points = o3d.utility.Vector3dVector(
    np.asarray(combined_mm.points) * 1000.0)        # m → mm

o3d.io.write_point_cloud("segmented_box_mm_ascii.pcd",
                         combined_mm,
                         write_ascii=True)

True

In [33]:
pcd2 = o3d.io.read_point_cloud("segmented_box_mm_ascii.pcd")
o3d.visualization.draw_geometries([pcd2])   # ← no custom camera kwargs

### Full correct code

In [27]:
import open3d as o3d
import numpy as np

MM_TO_M = 1/1000.0  # your file is in millimetres

def level_cloud(pcd, distance_thresh=0.005, num_iters=1000):
    # Find the dominant plane (your table)…
    (a, b, c, d), inliers = pcd.segment_plane(distance_thresh,
                                              ransac_n=3,
                                              num_iterations=num_iters)
    # normalise
    n = np.array([a, b, c], dtype=float)
    n /= np.linalg.norm(n)
    # build Rodrigues rotation sending n → [0,0,1]
    z = np.array([0, 0, 1.0])
    v = np.cross(n, z)
    s = np.linalg.norm(v)
    if s < 1e-8:
        R = np.eye(3)
    else:
        c = np.dot(n, z)
        vx = np.array([[   0, -v[2],  v[1]],
                       [ v[2],    0, -v[0]],
                       [-v[1],  v[0],   0]])
        R = np.eye(3) + vx + vx @ vx * ((1 - c) / (s**2))
    # rotate about the plane centroid so we don’t “lift” or “sink” the table
    centroid = np.asarray(pcd.points)[inliers[0]]
    return (pcd.translate(-centroid)
               .rotate(R, center=(0,0,0))
               .translate(centroid),
            R)

def measure_box(pcd_path,
                table_thresh_mm=5,
                lid_thresh_mm=3,
                dbscan_min=20):
    # 0) Load & convert → metres
    pcd = o3d.io.read_point_cloud(pcd_path)
    pts = np.asarray(pcd.points, float) * MM_TO_M
    pcd.points = o3d.utility.Vector3dVector(pts)

    # 1) Level the cloud so table ≡ perfect horizontal
    pcd, R = level_cloud(pcd,
                         distance_thresh=table_thresh_mm*MM_TO_M)

    # 2) Denoise & peel off the table (capture its plane model too)
    pcd, _       = pcd.remove_statistical_outlier(20, 2.0)
    (a,b,c,d_tab), in_tab = pcd.segment_plane(table_thresh_mm*MM_TO_M,
                                              ransac_n=3,
                                              num_iterations=1000)
    n_tab = np.array([a,b,c])
    n_tab /= np.linalg.norm(n_tab)
    d_tab /= np.linalg.norm([a,b,c])

    table_cloud = pcd.select_by_index(in_tab)
    obj_cloud   = pcd.select_by_index(in_tab, invert=True)

    # 3) Cluster the leftover points → pick the biggest chunk
    #    (this is your parcel)
    nn = obj_cloud.compute_nearest_neighbor_distance()
    eps = 2 * np.percentile(nn, 90)
    labels = np.array(obj_cloud.cluster_dbscan(eps, dbscan_min, False))

    if labels.max() >= 0:
        counts      = np.bincount(labels[labels>=0])
        main_label  = counts.argmax()
        parcel      = obj_cloud.select_by_index(
                          np.where(labels==main_label)[0])
    else:
        parcel = obj_cloud  # fallback, if DBSCAN fails

    # 4) XY dims from an Oriented Bounding Box on the leveled parcel
    obb = parcel.get_oriented_bounding_box()
    dx, dy, _ = obb.extent

    # 5) Fit the top-of-box plane and measure its offset from the table plane
    (a2,b2,c2,d_lid), in_lid = parcel.segment_plane(
                                   lid_thresh_mm*MM_TO_M, 3, 500)
    n_lid = np.array([a2,b2,c2])
    n_lid /= np.linalg.norm(n_lid)
    d_lid /= np.linalg.norm([a2,b2,c2])

    height_m = abs(d_lid - d_tab)

    # 6) Return everything in cm
    return (dx*100, dy*100, height_m*100), parcel, table_cloud, parcel.select_by_index(in_lid)

# ——— run it ———
dims_cm, parcel, tbl, lid = measure_box("PointCloud/PointCloud_box_a1.pcd")
print(f" L × W × H ≈ {dims_cm[0]:.1f} cm × {dims_cm[1]:.1f} cm × {dims_cm[2]:.1f} cm")


 L × W × H ≈ 20.5 cm × 10.9 cm × 7.5 cm


In [32]:
# 2. Color each cloud for clarity
table_pts.paint_uniform_color([1.0, 0.0, 0.0])   # red
parcel.paint_uniform_color([0.2, 0.8, 0.2])      # green
lid_pts.paint_uniform_color([0.0, 0.0, 1.0])     # blue

# 3. (Optional) draw the oriented bounding box around the parcel
obb = parcel.get_oriented_bounding_box()
obb.color = (1.0, 1.0, 0.0)                       # yellow

# 4. Visualize them all
o3d.visualization.draw_geometries([table_pts, parcel, lid_pts, obb],
                                  window_name="Box Measurement",
                                  width=800, height=600)

### Full code 2

In [61]:
import open3d as o3d
import numpy as np

MM_TO_M = 1/1000.0           # conversion factor: millimetres → metres

# ---------------------------------------------------------------------
def level_cloud(pcd: o3d.geometry.PointCloud,
                distance_thresh=0.002,  # 2 mm
                num_iters=1000):
    """Rotate the dominant plane in `pcd` to be horizontal (Z axis)."""
    (a, b, c, d), inliers = pcd.segment_plane(distance_thresh,
                                              ransac_n=3,
                                              num_iterations=num_iters)
    n = np.array([a, b, c], float)
    n /= np.linalg.norm(n)
    z = np.array([0, 0, 1], float)
    v = np.cross(n, z)
    s = np.linalg.norm(v)
    if s < 1e-6:
        R = np.eye(3)
    else:
        c_dot = np.dot(n, z)
        vx = np.array([[   0, -v[2],  v[1]],
                       [ v[2],    0, -v[0]],
                       [-v[1],  v[0],   0]], float)
        R = np.eye(3) + vx + (vx @ vx) * ((1 - c_dot) / (s**2))
    centroid = np.asarray(pcd.points)[inliers[0]]
    pcd2 = pcd.translate(-centroid, relative=False)
    pcd2 = pcd2.rotate(R, center=(0,0,0))
    pcd2 = pcd2.translate(centroid, relative=False)
    return pcd2

# ---------------------------------------------------------------------
def plane_fit(cloud: o3d.geometry.PointCloud, thresh: float, n_iter=1000):
    """Fit a plane to `cloud`. Return (normal, d, inlier_idx)."""
    (a, b, c, d), inliers = cloud.segment_plane(
        distance_threshold=thresh,
        ransac_n=3,
        num_iterations=n_iter
    )
    n = np.array([a, b, c], float)
    norm = np.linalg.norm(n)
    n /= norm
    d /= norm
    return n, d, inliers

# ---------------------------------------------------------------------
def measure_box_from_lid(pcd_path: str,
                         plane_thresh_m=0.005,  # 5 mm
                         min_points=20,
                         mm_to_m=1/1000.0):
    # 0) load & convert to metres
    pcd = o3d.io.read_point_cloud(pcd_path)
    pts = np.asarray(pcd.points, float) * mm_to_m
    pcd.points = o3d.utility.Vector3dVector(pts)

    # 1) level scene so table is flat
    pcd = level_cloud(pcd, distance_thresh=plane_thresh_m)

    # 2) clean noise
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20,
                                            std_ratio=2.0)

    # 3) fit & extract table plane
    n_tab, d_tab, tab_inliers = plane_fit(pcd, plane_thresh_m)
    table_pc = pcd.select_by_index(tab_inliers)
    obj_pc   = pcd.select_by_index(tab_inliers, invert=True)

    # 4) cluster → pick largest as box
    labels = np.array(obj_pc.cluster_dbscan(eps=0.02,
                                            min_points=min_points,
                                            print_progress=False))
    if np.any(labels >= 0):
        biggest = np.bincount(labels[labels >= 0]).argmax()
        box_pc = obj_pc.select_by_index(np.where(labels == biggest)[0])
    else:
        raise RuntimeError("No clusters found; adjust eps/min_points")

    # 5) fit lid (top) plane on the box
    n_lid, d_lid, lid_inliers = plane_fit(box_pc, plane_thresh_m)
    lid_pc = box_pc.select_by_index(lid_inliers)

    # 6) height = |d_lid - d_tab|
    height_m = abs(d_lid - d_tab)

    # 7) XY dimensions from the lid’s own OBB
    lid_obb = lid_pc.get_oriented_bounding_box()
    xy_m    = lid_obb.extent[:2]

    # 8) dimensions in cm: [L, W, H]
    dims_cm = np.array([xy_m[0], xy_m[1], height_m]) * 100.0

    return dims_cm, table_pc, box_pc, lid_pc, lid_obb

# ---------------------------------------------------------------------
if __name__ == "__main__":
    pcd_file = "PointCloud/PointCloud_box_a1.pcd"
    dims, table_pc, box_pc, lid_pc, lid_obb = measure_box_from_lid(pcd_file)
    print(f"Box ≈ {dims[0]:.1f} × {dims[1]:.1f} × {dims[2]:.1f} cm  (L×W×H)")

    # color for viz
    table_pc.paint_uniform_color([1.0, 0.0, 0.0])   # table = red
    lid_pc.paint_uniform_color([0.0, 1.0, 0.0])     # lid   = green
    box_pc.paint_uniform_color([0.2, 0.8, 1.0])     # box body = cyan
    lid_obb.color = (1.0, 1.0, 0.0)                 # lid OBB = yellow

    # visualize
    o3d.visualization.draw_geometries(
        [table_pc, lid_pc, box_pc, lid_obb],
        window_name="Table/red, Lid/green, Box body/cyan, Lid-OBB/yellow",
        width=800, height=600
    )

Box ≈ 20.8 × 11.5 × 7.8 cm  (L×W×H)


In [58]:
import open3d as o3d
import numpy as np

MM_TO_M = 1/1000.0    # mm → m
N_RUNS  = 15          # ensemble size

# ---------------------------------------------------------------------
def level_cloud(pcd: o3d.geometry.PointCloud,
                distance_thresh=0.002,
                num_iters=1000) -> o3d.geometry.PointCloud:
    """Rotate `pcd` so its dominant plane is horizontal."""
    (a, b, c, d), inliers = pcd.segment_plane(
        distance_threshold=distance_thresh,
        ransac_n=3,
        num_iterations=num_iters
    )
    # plane normal
    n = np.array([a, b, c], float)
    n /= np.linalg.norm(n)
    # desired vertical
    z = np.array([0, 0, 1], float)
    # rotation axis & angle
    v = np.cross(n, z)
    s = np.linalg.norm(v)
    if s < 1e-6:
        R = np.eye(3)
    else:
        c_dot = np.dot(n, z)
        vx = np.array([[   0, -v[2],  v[1]],
                       [ v[2],    0, -v[0]],
                       [-v[1],  v[0],   0]], float)
        R = np.eye(3) + vx + (vx @ vx) * ((1 - c_dot) / (s**2))
    # rotate around any inlier to keep plane in place
    centroid = np.asarray(pcd.points)[inliers[0]]
    pcd2 = pcd.translate(-centroid, relative=False)
    pcd2 = pcd2.rotate(R, center=(0, 0, 0))
    pcd2 = pcd2.translate(centroid, relative=False)
    return pcd2

# ---------------------------------------------------------------------
def plane_fit(cloud: o3d.geometry.PointCloud,
              thresh: float,
              n_iter=1000):
    """RANSAC-fit a plane, return (unit-normal, d, inlier-indices)."""
    (a, b, c, d), inliers = cloud.segment_plane(
        distance_threshold=thresh,
        ransac_n=3,
        num_iterations=n_iter
    )
    n = np.array([a, b, c], float)
    norm = np.linalg.norm(n)
    n /= norm
    d /= norm
    return n, d, inliers

# ---------------------------------------------------------------------
def measure_box_from_lid(pcd: o3d.geometry.PointCloud,
                         plane_thresh_m=0.005,
                         min_points=20):
    """
    Given a leveled & cleaned cloud, segment table & box, fit lid plane,
    and return dims_cm [L, W, H], plus table_pc, box_pc, lid_pc, lid_obb.
    """
    # 1) Table plane
    n_tab, d_tab, tab_inliers = plane_fit(pcd, plane_thresh_m)
    table_pc = pcd.select_by_index(tab_inliers)
    obj_pc   = pcd.select_by_index(tab_inliers, invert=True)

    # 2) Box cluster
    labels = np.array(obj_pc.cluster_dbscan(eps=0.02,
                                            min_points=min_points,
                                            print_progress=False))
    if np.any(labels >= 0):
        idx = np.bincount(labels[labels>=0]).argmax()
        box_pc = obj_pc.select_by_index(np.where(labels == idx)[0])
    else:
        raise RuntimeError("No clusters found")

    # 3) Lid plane
    n_lid, d_lid, lid_inliers = plane_fit(box_pc, plane_thresh_m)
    lid_pc = box_pc.select_by_index(lid_inliers)

    # 4) Height
    height_m = abs(d_lid - d_tab)

    # 5) XY from lid OBB
    lid_obb = lid_pc.get_oriented_bounding_box()
    xy_m    = lid_obb.extent[:2]

    # 6) dims in cm
    dims_cm = np.array([xy_m[0], xy_m[1], height_m]) * 100.0
    return dims_cm, table_pc, box_pc, lid_pc, lid_obb

# ---------------------------------------------------------------------
def main(pcd_file: str):
    # load & scale once
    raw = o3d.io.read_point_cloud(pcd_file)
    pts = np.asarray(raw.points, float) * MM_TO_M
    raw.points = o3d.utility.Vector3dVector(pts)

    dims_list = []
    last_out  = None

    for i in range(N_RUNS):
        # level & denoise
        pcd = level_cloud(raw, distance_thresh=0.005)
        pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20,
                                                std_ratio=2.0)

        # measure
        dims_cm, table_pc, box_pc, lid_pc, lid_obb = measure_box_from_lid(pcd)
        dims_list.append(dims_cm)
        last_out = (table_pc, box_pc, lid_pc, lid_obb)
        print(f"Run {i+1}: {dims_cm[0]:.1f} × {dims_cm[1]:.1f} × {dims_cm[2]:.1f} cm")

    # ensemble: robust outlier removal via MAD
    all_dims = np.vstack(dims_list)
    med      = np.median(all_dims, axis=0)
    mad      = np.median(np.abs(all_dims - med), axis=0)

    mask = np.ones(len(all_dims), dtype=bool)
    for j in range(3):
        if mad[j] > 1e-6:
            mask &= np.abs(all_dims[:, j] - med[j]) <= 3 * mad[j]

    filtered = all_dims[mask]
    if filtered.size == 0:
        filtered = all_dims

    mean_dims = filtered.mean(axis=0)
    std_dims  = filtered.std(axis=0)
    num_used  = filtered.shape[0]

    print(f"\nEnsemble over {N_RUNS} runs ({num_used} used):")
    print(f"  Mean: {mean_dims[0]:.1f} × {mean_dims[1]:.1f} × {mean_dims[2]:.1f} cm")
    print(f"  Std:  {std_dims[0]:.1f}, {std_dims[1]:.1f}, {std_dims[2]:.1f} cm")

    # visualize last run
    table_pc, box_pc, lid_pc, lid_obb = last_out
    table_pc.paint_uniform_color([1, 0, 0])
    lid_pc.paint_uniform_color([0, 1, 0])
    box_pc.paint_uniform_color([0.2, 0.8, 1])
    lid_obb.color = (1, 1, 0)

    o3d.visualization.draw_geometries(
        [table_pc, lid_pc, box_pc, lid_obb],
        window_name="Last run visualization",
        width=800, height=600
    )

if __name__ == "__main__":
    main("PointCloud/PointCloud_box_a1.pcd")


Run 1: 20.4 × 10.3 × 7.8 cm
Run 2: 20.3 × 10.3 × 8.6 cm
Run 3: 20.6 × 10.6 × 7.9 cm
Run 4: 75.4 × 34.8 × 0.7 cm
Run 5: 76.8 × 49.9 × 1.1 cm
Run 6: 20.3 × 10.2 × 8.6 cm
Run 7: 20.7 × 10.6 × 7.8 cm
Run 8: 20.9 × 12.1 × 8.6 cm
Run 9: 21.1 × 12.2 × 7.8 cm
Run 10: 20.6 × 10.7 × 8.6 cm
Run 11: 20.9 × 11.8 × 7.8 cm
Run 12: 20.6 × 10.6 × 8.6 cm
Run 13: 20.7 × 11.5 × 7.7 cm
Run 14: 20.8 × 10.6 × 8.7 cm
Run 15: 20.7 × 11.7 × 7.7 cm

Ensemble over 15 runs (12 used):
  Mean: 20.6 × 10.9 × 8.2 cm
  Std:  0.2, 0.6, 0.4 cm
